<table style="width:100%"><tr>
<td>Technische Universität Berlin<br>
Electrical Engineering and Computer Science<br>
    Internet of Things for Smart Buildings <br>
Prof. Dr. Sergio Lucia, Felix Fiedler, Benjamin Karg </td>
<td>  <img src="logo_tu.png" style="width: 20%;" align="right"/> </td>
</tr>
</table>

***
**Model Predictive Control 2020**
***

 <h1 style="font-size:40px;">Project 2B - Discretization Schemes (Kite)</h1> 
 
Julia Graham, Tor Istvan Stadler Kjetså, Raschid Younes 
 <hr>


### **Abstract**

The usage of wind energy as a power source is a familiar idea, and for the most part utilized with windmills. At higher altidtudes, the wind is much stronger and less fluctuating. Airplanes usually take advantage of this when travelling. Structures such as windmills can not be build so tall that they can take advantage of these strong winds, but a kite fastened to the ground with a tether long enough can do exactly that. There are different techniques on how to convert the force working on the kite into electrical power - however, in this project, the power from the kite is meant to pull the boat and reduce fuel consumption, and the pull from the kite can be applied with significantly less loss.

## **1. Introduction** <a class="anchor" id="intro"></a>
### **1.1 Problem Statement** <a class="anchor" id="problem"></a>

- Pull a boat to reduce fuel consumption &rightarrow; maximize tension

- Continuous-time nonlinear Problem &rightarrow; Nonlinear MPC



<table style="width:100%"><tr>
<td>
<ul style="font-size:14px"> <b>Discretization schemes</b>:
   <li>Implicit Euler</li>
   <li>Orthogonal collocation</li>
   <li>Single Shooting</li>
</ul>    
</td>
  <td>   
    <figure>
      <img src="boat_kite.png" alt="drawing" width="400"/>
      <figcaption> Fig.1: SkySails towing kite system, see [1]</figcaption>
    </figure>  
  </td>
</tr>
</table>






<p style="font-size:10px">[1] M. Erhard and H. Strauch, “Control of towing kites for seagoing vessels,” IEEE Trans-
actions on Control Systems Technology, vol. 21, pp. 1629–1640, 2013.</p>


### **1.2 Model** <a class="anchor" id="model"></a>


<table style="width:100%"><tr>
<td>
<ul style="font-size:14px" > <b>Assumptions</b>:
   <li>Kite has no mass</li>
   <li>Tether of fixed length</li>
   <li>Kite's inertia is negligible</li>
   <li>All states can be measured</li>
   <br>
   <b>states</b> 
    $x = [\theta \ \phi \ \psi]^T$ 
   <li>$\theta$ ( zenith angle)</li>
   <li>$\phi$ (azimuth angle)</li>
   <li>$\psi$ (orientation angle)</li>
</ul>
    
</td>
  <td>   
    <figure>
      <img src="kite_coordinate_system.png" alt="drawing" width="300"/>
      <figcaption> Fig.2: Coordinate system, see [1]</figcaption>
    </figure>  
  </td>
</tr>
</table>









<table style="width:100%"><tr>
<td>
<ul style="font-size:14px" > <b>State Dymamics</b>:
$$ \begin{align}
\dot{\theta} &= \frac{v_a}{L}(\cos \psi - \frac{\tan \theta }{E}) \\
\dot{\phi} &= - \frac{v_a}{L \sin \theta} \sin \psi \\
\dot{\psi} &= \frac{v_a}{L} \tilde{u} + \dot{\phi} \cos \theta \end{align}$$
   <br>
    <br>
$$ \dot{x} = f(x,u) = 
\begin{pmatrix}
\frac{v_a}{L}(\cos \psi - \frac{\tan \theta }{E}) \\
- \frac{v_a}{L \sin \theta} \sin \psi \\
\frac{v_a}{L} \tilde{u} + \dot{\phi} \cos \theta
\end{pmatrix},\quad \quad x = 
\begin{pmatrix}
\theta \\
\phi \\
\psi 
\end{pmatrix}
$$
</ul>
    
</td>
  <td>   
    <figure>
      <img src="kite_structure.png" alt="drawing" width="300"/>
      <figcaption> Fig.3: geometry of steering the kite, see [1]</figcaption>
    </figure>  
  </td>
</tr>
</table>



### **1.3 Trajectory Example** <a class="anchor" id="trajectory"></a>
<figure>
  <img src="8_trajectory.png" alt="drawing" width="800"/>
  <figcaption> Fig.4: kite trajectory example with minimum height constraints</figcaption>
</figure>  

## **2. Methodology** <a class="anchor" id="method"></a>


### **2.1 MPC Problem Formulation** <a class="anchor" id="mpc_prob"></a>

The stage cost $L(x,u)$ contains the thrust $T_F$ which should be maximized and the second term describes the changes of the control input which should be minimized and is assigned a higher weight $w_{u}$ in order to obtain a smoothing trajectory. The signs represent here, if the term should be maximized or minimized.
$$ L(x,u) = - w_F T_F + w_u (\tilde{u} - \tilde{u_{old}})^2 $$

The most critical constraint is the height which is described by

$$ h(x) = L \sin \theta \cos \phi $$

The height will thus decrease as the kite moves in the direction of the wind, decreasing $\theta$ from an upright position, as well as in the direction normal to the wind, in the y-direction, such that $\phi$ increases.

The general discrete optimization problem should look like the following: 

$$\begin{align} &\mathop{\mathrm{minimize}}_{u(k)} &\sum_{k=0}^{N-1} L\big(x(k),u(k)\big)\\
&\text{subject to}
& x_{k+1} = f(x(k),u(k)), \ x(0) = x_0 \\
& & h(x) \geq h_{min}\\
& & u(k) \in \mathbb{U}, \ x(k) \in \mathbb{X} \\
& & k \in [0,N-1]
\end{align}$$

Solving this optimal control problem online for each sampling instant, and applying the first control of the optimal control input sequence for each iteration, the problem becomes the MPC problem formulation.



### **2.2 Discretization Methods** <a class="anchor" id="discr_method"></a>

#### **2.2.1 Implicit Euler** <a class="anchor" id="impl_eu"></a>
Implicit Euler is a first-order numerical procedure for solving ODEs. In each time step, an equation is solved to obtain the next state $x_{k+1}$. The formulation is implicit since the next state appears on both sides on the equation. It is formulated as follows:
$$x_{k+1} = x_{k} + \Delta t f(x_{k+1}, u_k)$$

Where $\Delta t$ is the stepsize.

Implicit Euler could be used as a full discretization scheme, i.e, discretizing both states and the control input. By dividing the horizon into N control stages and in each subinterval discretize the inputs and states, where the states are discretized as seen above. We can then replace the system ODE by finitely many equality constraints, since we get an equality constrain for each time step. The formulation of the optimal control problem with the discretized stage cost would then become as follows:

$$\begin{align} &\mathop{\mathrm{minimize}}_{u(k){x(k)}} &\sum_{k=0}^{N-1}  \Delta t l\big(x(k),u(k)\big) \\
&\text{subject to}
& x_{k+1} = x_{k} + \Delta t f(x_{k+1}, u_k), \ x(0) = x_0 \\
& & \\
& &  \\
& &
\end{align}$$


#### **2.2.2 Single Shooting** <a class="anchor" id="single_shooting"></a>

In Direct Single Shooting the horizon is divided into N control states. The control trajectory is then parametrized within each subinterval, either as a piecewise constant function, or as a piecewise linear function. The task is then to find the optimal control input sequence {$u_{k}$} that minimizes the cost function formulated below by solving the optimization problem for each iteration. The final optimization problem following the single shooting scheme is formulated as follows:

$$\begin{align} &\mathop{\mathrm{minimize}}_{u(k)} &\sum_{k=0}^{N-1} \int_{t_k}^{t_{k+1}} L\big(x(t),u(k)\big) \\
&\text{subject to}
& x_{k+1} = x_{k} + \int_{t_{k}}^{t_{k+1}} f(x(t),u(k)), \ x(0) = x_0 \\
& & h(x(k)) \geq h_{min}\\
& & u(k) \in \mathbb{U}, \ x(k) \in \mathbb{X} \\
& &
\end{align}$$

To obtain the next state, the current state is added to the system integrated from the current time step to the next time step as seen in the constraints above. The state integral is solved by the numerical integrator that is used. This integrator is used to compute all continuous operations such as the integrals for each time step in the cost function, in the state constraints as well as compute the derivatives of the cost function which is needed for the optimization problem in each iteration.

CasADi comes with a set of ODE solvers in the SUNDIALS suite. A popular one is CVOdes and would be a suitable numerical integrator to use in this implementation.

In order to deal with continuous state constraints, they are checked only at certain time steps to make sure they are not violated which is what will be done for the height constraint as seen above.


#### **2.2.3 Orthogonal Collocation** <a class="anchor" id="orth_coll"></a>
Orthogonal collocation (on finite elements) is a discretization technique where the states are approximated using interpolated polynomials:


$$\begin{equation}
x(t) \approx x_k(t) := \sum_{j=0}^{ndeg}x_k^jl^j\\
l^i(t) = {\prod_{j=0\\j \neq i}^{ndeg}}\frac{t-t_j}{t_i-t_j}
\end{equation}$$


where $x_k^j$ is defined by the $j^{th}$ collocation point of the $k^{th}$ element and $l^i(t)$ is the Lagrange polynomial for the $i^{th}$ collocation point at time $t$.

Using this, the differential equation $ \dot{x} = f(x, u) $ can be replaced by: 


$$
\begin{equation}
\sum_{i=0}^{ndeg} x_k^i\frac{\partial l^i(t_j)}{\partial t} = f(x_k^j, u_k^j)
\end{equation}
$$


Computing the Lagrange polynomials and their time derivatives results in an approximation, usually with high accuracy and few variables.


The static optimization problem to be solved is then given by

$$
\begin{align}
&\mathop{\mathrm{minimize}}_{u(k), x_k^j} &\sum_{k=0}^{N-1}l(x(k), u(k)) + V_f(x(N)) \\
&\text{subject to} &\sum_{i=0}^{ndeg}x_k^i \frac{\partial l^i(t_k^j)}{\partial t} = f(x_k^j, u(k)), \forall k=0,..., N-1, j=0,...,ndeg \\
& & h(x(k)) \geq h_{min}\\
& & u(k) \in \mathbb{U}, \ x(k) \in \mathbb{X} \\
\end{align}
$$


#### **2.2.4 Initial Hypotheses** <a class="anchor" id="hypotheses"></a>

**Implicit Euler** 
Implicit Euler is a simple and easy method to implement. However, since the problem is nonlinear, a sufficiently small stepsize is needed to accurately model the dynamics. This implies a lot of constraints and thus a large optimization problem with may decision variables. We would also have a problem with smoothness of the trajectory with this solution. As a sufficiently accurate model would result in a very small stepsize, the optimization problem would become very big which in turn would take a lot of computation time for each iteration, which again would not contribute to the desired smooth behavior of the kite.

We therefore expect this method to perform poorly, and that it will have difficulty even with tuning of parameters like the stepsize and the sampling time of the controller $t_{c}$.


**Single shooting** is easy to implement and with this scheme the only optimization variables becomes the $u_{k}$ works best on problems where the control is very simple. In this project the control input $\tilde{u}$ affects the orientation of the kite, which then again affects the zenith and the azimuth angle indicating the kite's position, and is not too simple. The approximation of modeling the input as a constant or linear function in each time step is therefore a bad approximation.

Also due to the integrator embedding, the single shooting method can be inefficient and result in a longer computation time for each step, and thus slower sampling frequency. This is likely to affect the optimality of the solution.

**Orthogonal collocation** can 
is a good choice for obtaining high-accuracy solutions to problems where the accuracy of the control trajectory is important. Path constraints might cause problems, so this technique works best when the optimal solution is smooth, which we expect it to be in our case, as rapid changes in the kite's orientation can cause unpredictable behaviour. 


## **3. Preliminary Results** <a class="anchor" id="prem_res"></a>

We've started development of an MPC with orthogonal collocation as discretization method.  
Using:


        # prediction horizon 
        N = 50
        # collocation degree K = 3
        # yielding: 
        tau_col = [0]+collocation_points(K, 'legendre')
        # Simulation steps
        N_sim = 200
        # sampling time, t_c
        dt = 0.2
        
has resulted in fairly promising results.






<figure>
  <img src="orth_col_default_setup.png" alt="orthogonal_collocation_trajectory" width="600"/>
  <center><figcaption> Fig.5: Obtained kite trajectory with orthognal collocation 3D-space</figcaption></center>
</figure>  




<figure>
  <img src="orth_col_K=3_N=50_col_points=legendre_dt=0.2.png" alt="orthogonal_collocation_plots" width="1000"/>
  <center><figcaption> Fig.6: Obtained kite states, inputs and costs with orthognal collocation, $t_c = 0.2s$</figcaption></center>
</figure>  

Using 'Legendre' collocation points instead of 'Radau' hasn't given much different results. In some cases the trajectory has appeared a bit smoother, while in other cases it has appeared less responsive. The computation time $t_{sol}$ appears to be slightly smaller with 'Legendre', however.  

The boundary constraints on $\psi \in [0, 2\pi]$ caused some problems as the kite wasn't able to turn back around, but changing these to $\psi \in [-\pi, \pi]$ solved this.


The main problem that we're struggling with right now is that the computation time for each step of MPC, $t_{sol}$ is larger than the sampling time, $t_c$.  
With $t_c=0.2s$ the avarage computation time lies around $\overline{t_{sol}} \approx 0.06s$, but there is usually a couple of spikes resulting in a maximum computation time of $ t_{sol_{max}} \approx 0.31s$.  Increasing $t_c$ to $t_c = 0.4s$ doesn't only reduce the performance of the MPC, it also results in even larger spikes with $t_{sol_{max}} \approx 3.8s$ for some reason we have yet to figure out.

<figure>
  <img src="orth_col_K=3_N=50_col_points=legendre_dt=0.4.png" alt="orthogonal_collocation_plots" width="1000"/>
  <center><figcaption> Fig.7: Obtained kite states, inputs and costs with orthognal collocation, $t_c = 0.4 s$</figcaption></center>
</figure>